In [1]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('data_cleaned_step_1.csv')

In [4]:
df.head()

,Unnamed: 0,System,Tag,Excursion,ECount,Follow-up,Comments,Excursion details,Analysis,Recommendation
0,0,Oil Separation System,USN_5LI200304.PV,!st stage separator Train B oil Level went upt...,2,No Notification Required,Excursion details:-!st Stage Separator train B...,:-!st Stage Separator train B oil level went m...,:- Level control valve was 100% open working ...,:- If feasible line up 34' Bypass in line wh...
1,1,Oil Separation System,USN_5PIC200302A.PV,!st stage separator Train B Pressure went high...,1,No Notification Required,Excursion details:-1st stage separator pressur...,:-1st stage separator pressure went high upto ...,:- Vessel pressure went more than two time of ...,:- Check the flare knock out drum level.
2,2,Water injection Pumps,USN_5FI540205C.PV,Main Water injection pump suction flowrate is ...,1,No Notification Required,Excursion details:-Main Water Injection pump 5...,:-Main Water Injection pump 5P5402C suction fl...,:- Over the past 1 month the main wtaer injec...,:- Flush the transmitter if poosiible
3,3,HP compressor,USN_5LI310512A.PV,HP1 ompressor Suction Scrubber level transmitt...,1,Notification Req'd,Excursion details:-HP1 Compreesor Suction scru...,:-HP1 Compreesor Suction scrubber level transm...,:- HP1 Scrubber level transmitter value is a...,:- To safeguard the gas compressor KA 3105A fr...
4,4,TEG A,USN_5TI300602B-S.PV,ES3006B Reflux condenser top temperature trans...,1,Notification Req'd,Excursion details:-TEG train B reboiler reflux...,:-TEG train B reboiler reflux drum top tempera...,:-TEG train B reboiler reflux drum top tempera...,:- The temperature transmitter 5TI300602B need...


In [7]:
df['Excursion details']=df['Excursion details'].str.replace(':-','')
df['Analysis']=df['Analysis'].str.replace(':-','')
df['Recommendation']=df['Recommendation'].str.replace(':-','')

In [9]:
df['Excursion details']=df['Excursion details'].str.replace(':','')
df['Analysis']=df['Analysis'].str.replace(':','')
df['Recommendation']=df['Recommendation'].str.replace(':','')

In [10]:
df

,Unnamed: 0,System,Tag,Excursion,ECount,Follow-up,Comments,Excursion details,Analysis,Recommendation
0,0,Oil Separation System,USN_5LI200304.PV,!st stage separator Train B oil Level went upt...,2,No Notification Required,Excursion details:-!st Stage Separator train B...,!st Stage Separator train B oil level went mor...,Level control valve was 100% open working as...,If feasible line up 34' Bypass in line when...
1,1,Oil Separation System,USN_5PIC200302A.PV,!st stage separator Train B Pressure went high...,1,No Notification Required,Excursion details:-1st stage separator pressur...,1st stage separator pressure went high upto 11...,Vessel pressure went more than two time of se...,Check the flare knock out drum level.
2,2,Water injection Pumps,USN_5FI540205C.PV,Main Water injection pump suction flowrate is ...,1,No Notification Required,Excursion details:-Main Water Injection pump 5...,Main Water Injection pump 5P5402C suction flow...,Over the past 1 month the main wtaer injecti...,Flush the transmitter if poosiible
3,3,HP compressor,USN_5LI310512A.PV,HP1 ompressor Suction Scrubber level transmitt...,1,Notification Req'd,Excursion details:-HP1 Compreesor Suction scru...,HP1 Compreesor Suction scrubber level transmit...,HP1 Scrubber level transmitter value is at ...,To safeguard the gas compressor KA 3105A from...
4,4,TEG A,USN_5TI300602B-S.PV,ES3006B Reflux condenser top temperature trans...,1,Notification Req'd,Excursion details:-TEG train B reboiler reflux...,TEG train B reboiler reflux drum top temperatu...,TEG train B reboiler reflux drum top temperatu...,The temperature transmitter 5TI300602B need t...
5,5,TEGA,USN_5TI300431A-S.PV,ES3008A Lean glycol cold cooler suspected chok...,1,Notification Req'd,Excursion details:-ES3008A Lean glycol cold co...,ES3008A Lean glycol cold cooler suspected chok...,As it is evident from trend that lean glycol c...,Hotter TEG will increase TEG vaporization los...
6,6,Crude Oil heater,USN_5PDI200127B,Crude Oil heater ES2001 B PDIT is not working ...,3,Notification Req'd,Excursion details:-ES2001B Crude Oil heater DP...,ES2001B Crude Oil heater DP transmitter USN_5P...,As it is being evident from Seeq trend crude ...,If possible kindly flush both the transmitte...
7,7,Teg Gas Filter Coalescer,VF3003A PDIT 5PDI300304A,Teg Gas filter coalescer VF3003A DP is high,5,Notification Req'd,Excursion details:-TEG Gas Coalescer Filter VF...,"TEG Gas Coalescer Filter VF3003A Dp is high,PD...",PDIT 5PDI300304A is constantly reading more ...,As it is seen from P&ID that coalescer fliter...
8,8,Ist Stage Separator B Pressure Control valve,USN_5PIC200302A.OUT,Ist Stage Separator B Pressure Control valve O...,1,Notification Req'd,Excursion details:-Ist Stage Separator B Press...,Ist Stage Separator B Pressure Control valve U...,-Ist Stage Separator B Pressure Control valve...,If control valve 5PV12001302 get calibrated ...
9,9,MP compressor Train B Discharge temperature,USN_5TI2-310433B-S.PV,MP compressor B Discharge temperature is near ...,5,Notification Req'd,Excursion details:-MP compressor B KA 3104B D...,MP compressor B KA 3104B Discharge temperatur...,-MP compressor B Discharge temperature is nea...,If we able to keep the gas temperature in co...


In [19]:
input_texts=(df['Excursion details'] + df['Analysis']).tolist()

In [20]:
input_texts

['!st Stage Separator train B oil level went more than high high limit 2 to 3 time i.e. more than 89.5% after startup.\r Level control valve was 100% open  working as per level control philosphy  but more flow can be achieved through bypass\r',
 '1st stage separator pressure went high upto 11 bar though set point was 5 bar\r Vessel pressure went more than two time of set pressure(5 Bar) and flare valve opened till 100% it may carry over the liquid into flare line.\r',
 "Main Water Injection pump 5P5402C suction flowrate is showing less value than trip set point i.e.350m3/hr in suction line  and flowmeter tag is also not matching with P&ID\r Over the past 1  month the main wtaer injection pump 5P5402 C main suction flow is not showing correct value.Trip set point for suction flow is 230 m3/hr where as it's showing 73 m3/hr\r",
 'HP1 Compreesor Suction scrubber level transmitter USN_5LI310512A. is faulty ,\r HP1 Scrubber level transmitter  value is  at 100% since few days . Asset confirm

In [21]:
output_texts=df['Recommendation'].tolist()

In [22]:
output_texts

[" If feasible line up  34' Bypass  in line whenevr level build up suddenly in any of two 1st stage  separator train A or B ",
 ' Check the flare knock out drum level.',
 ' Flush the transmitter if poosiible ',
 ' To safeguard the gas compressor KA 3105A from condensate or any other liquid it is recommended to monitor scrubber(VZ3005A) level closely and if required drain the scrubber level manually .',
 ' The temperature transmitter 5TI300602B need to  be attended before TEG train B is taken into line to avoid any opeartional issue .',
 ' Hotter TEG will increase TEG vaporization losses with the dry gas. In addition, if TEG is above  54 C. the absorption process is not as efficient and may increase gas temperature such that the gas may not release water as readily, again contributing to less water absorption in final stages So for better cooling cooler cleaning required or lean glycol inlet  flow can be reduced in process side as it is  already flowing near to design flow (8.5m3/hr) to

In [23]:
# Tokenize input and output texts
tokenizer_inputs = Tokenizer()
tokenizer_inputs.fit_on_texts(input_texts)
input_sequences = tokenizer_inputs.texts_to_sequences(input_texts)

In [41]:
tokenizer_outputs = Tokenizer()
tokenizer_outputs.fit_on_texts(output_texts)
# Add special tokens to vocabulary
tokenizer_outputs.word_index['<sos>'] = len(tokenizer_outputs.word_index) + 1
tokenizer_outputs.word_index['<eos>'] = len(tokenizer_outputs.word_index) + 2

output_sequences = tokenizer_outputs.texts_to_sequences(output_texts)

In [43]:
tokenizer_outputs.word_index

{'the': 1,
 'to': 2,
 'of': 3,
 'is': 4,
 'and': 5,
 'in': 6,
 'it': 7,
 'check': 8,
 'for': 9,
 'be': 10,
 'if': 11,
 'transmitter': 12,
 'suggested': 13,
 'any': 14,
 'or': 15,
 'level': 16,
 'gas': 17,
 'as': 18,
 'also': 19,
 '\r': 20,
 'control': 21,
 'valve': 22,
 'healthiness': 23,
 'engineer': 24,
 'teg': 25,
 'will': 26,
 'water': 27,
 'temperature': 28,
 'flow': 29,
 'can': 30,
 'filter': 31,
 'asset': 32,
 'line': 33,
 'flare': 34,
 'need': 35,
 'confirm': 36,
 'same': 37,
 'recommended': 38,
 'avoid': 39,
 'with': 40,
 'not': 41,
 'better': 42,
 'kindly': 43,
 'heat': 44,
 'then': 45,
 'needs': 46,
 'functionality': 47,
 'informed': 48,
 'b': 49,
 'out': 50,
 'drum': 51,
 'from': 52,
 'may': 53,
 'that': 54,
 'so': 55,
 'usn': 56,
 'observe': 57,
 'hot': 58,
 'contactor': 59,
 'on': 60,
 'this': 61,
 'field': 62,
 'regarding': 63,
 'up': 64,
 'flush': 65,
 'compressor': 66,
 'scrubber': 67,
 'closely': 68,
 'process': 69,
 'glycol': 70,
 'side': 71,
 'which': 72,
 'degc': 7

In [44]:
max_encoder_seq_length = max(len(seq) for seq in input_sequences)
max_decoder_seq_length = max(len(seq) for seq in output_sequences)

In [45]:
# Pad sequences to uniform length
encoder_input_data = pad_sequences(input_sequences, maxlen=max_encoder_seq_length, padding='post')
decoder_input_data = pad_sequences(output_sequences, maxlen=max_decoder_seq_length, padding='post')


In [46]:
# Shift decoder output by one timestep
decoder_target_data = np.zeros_like(decoder_input_data)
decoder_target_data[:, 0:-1] = decoder_input_data[:, 1:]

In [54]:
# Define model architecture
latent_dim = 256
encoder_inputs = Input(shape=(None,))
encoder = LSTM(latent_dim, return_state=True,return_sequences=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]


ValueError: Input 0 of layer "lstm_7" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, None)

In [52]:

decoder_inputs = Input(shape=(None,))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(len(tokenizer_outputs.word_index) + 1, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

NameError: name 'encoder_states' is not defined